In [ ]:
import pandas as pd
ratings_data = pd.read_csv('./podaci/rating.csv')
movies_data = pd.read_csv('./podaci/movie.csv')

In [ ]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity

# Primjer DataFrame-a sa žanrovima filmova
movies_data = pd.DataFrame({
    'movieId': [1, 2, 3],
    'title': ['Movie A', 'Movie B', 'Movie C'],
    'genres': ['Action|Adventure|Sci-Fi', 'Comedy|Drama', 'Action|Drama|Thriller']
})

# Primjer DataFrame-a sa ocjenama korisnika
ratings_data = pd.DataFrame({
    'userId': [1, 2, 3],
    'movieId': [1, 2, 3],
    'rating': [5, 4, 3]
})

# Novi film koji želimo predviđati
new_movie = pd.DataFrame({
    'movieId': [4],
    'title': ['New Movie'],
    'genres': ['Action|Adventure']
})

# Funkcija za izračunavanje sličnosti između novog filma i postojećih filmova
def predict_average_rating(new_movie, movies_data, ratings_data):
    # Dodajemo novi film u skup podataka
    movies_data = pd.concat([movies_data, new_movie])

    # Stvaranje matrice ocjena
    user_movie_matrix = pd.pivot_table(ratings_data, values='rating', index='userId', columns='movieId', fill_value=0)

    # Računanje sličnosti između filmova pomoću kosinusne sličnosti
    movie_similarity = cosine_similarity(user_movie_matrix.T)

    # Izdvajamo vektor žanrova za novi film
    new_movie_genres = new_movie['genres'].str.get_dummies().reindex(columns=user_movie_matrix.columns, fill_value=0)

    # Računamo sličnost između novog filma i postojećih filmova
    new_movie_similarity = cosine_similarity(new_movie_genres, user_movie_matrix.T)

    # Računamo ponderiranu sličnost između novog filma i ocijenjenih filmova
    weighted_similarity = np.dot(new_movie_similarity, user_movie_matrix) / np.sum(np.abs(new_movie_similarity))

    # Računamo predviđene ocjene za novi film
    predicted_ratings = np.dot(weighted_similarity, ratings_data.set_index('movieId')['rating']) / np.sum(np.abs(weighted_similarity))
    print(predicted_ratings)

    return predicted_ratings[new_movie['movieId'].values[0]]

# Predviđanje prosječne ocjene za novi film
predicted_rating = predict_average_rating(new_movie, movies_data, ratings_data)

# Prikaz rezultata
print(f"Predicted average rating for '{new_movie['title'].values[0]}': {predicted_rating:.2f}")


In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Dodajte izmišljeni film za koji želite predviđati ocjene
new_movie_id = 99999

# Pretvorba podataka u DataFrame
new_movie_data = pd.DataFrame({'movieId': [99999], 'title': 'test', 'genres': 'Action|Comedy|Drama'})

# Dodavanje novih ocjena u skup podataka
movies_data = pd.concat([movies_data, new_movie_data], ignore_index=True)

# Izračun sličnosti između filmova koristeći cosine similarity
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)
cosine_sim = cosine_similarity(user_movie_ratings)

# Pronalaženje najsličnijih filmova novom filmu
similar_movies = cosine_sim[0]

# Izračun težinske srednje ocjene koristeći slične filmove
weighted_sum = 0
total_weight = 0

for i, similarity in enumerate(similar_movies):
    if similarity > 0:  # Zanemarujemo neslične filmove
        weighted_sum += similarity * user_movie_ratings.iloc[:, i].mean()
        total_weight += similarity

predicted_rating = weighted_sum / total_weight if total_weight > 0 else None

print(f'Predicted average rating for the new movie: {predicted_rating}')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Funkcija za određivanje sličnosti između korisnika koristeći cosine similarity
def find_similar_users(user_id, ratings_matrix):
    user_ratings = ratings_matrix.loc[user_id].values.reshape(1, -1)
    similarity = cosine_similarity(ratings_matrix, user_ratings)
    similar_users = pd.DataFrame(similarity, index=ratings_matrix.index, columns=['Similarity']).sort_values(by='Similarity', ascending=False)
    return similar_users

# Pretvorba podataka u matricu ocjena korisnika
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Pronalaženje sličnih korisnika za određenog korisnika (primjer: korisnik s ID-om 1)
target_user_id = 1
similar_users = find_similar_users(target_user_id, user_movie_ratings)

# Ispis sličnih korisnika
print(f'Similar users for user {target_user_id}:\n{similar_users.head()}')

In [ ]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

# Pretpostavljamo da imate DataFrame s podacima o ocjenama
# ratings_data = ...

# Funkcija za dobivanje preporuka za korisnika na temelju sličnih korisnika
def get_movie_recommendations(user_id, ratings_matrix, similar_users, n_recommendations=5):
    user_ratings = ratings_matrix.loc[user_id].values.reshape(1, -1)
    
    # Izračun sličnosti između korisnika i ocjena korisnika za filmove
    similarity = cosine_similarity(ratings_matrix, user_ratings)
    
    # Uzimanje težinskih ocjena korisnika za filmove sličnih korisnika
    weighted_ratings = similarity.T @ ratings_matrix.values
    
    # Normalizacija težinskih ocjena
    normalized_ratings = weighted_ratings / (1e-10 + abs(similarity).sum(axis=0))
    
    # Pronalaženje filmova koje ciljni korisnik još nije ocijenio
    unrated_movies = ratings_matrix.loc[user_id][ratings_matrix.loc[user_id].isna()].index
    
    # Sortiranje preporučenih filmova prema ocjenama
    recommendations = pd.DataFrame(normalized_ratings[:, unrated_movies], index=ratings_matrix.index, columns=unrated_movies)
    top_recommendations = recommendations.sum().sort_values(ascending=False).head(n_recommendations)
    
    return top_recommendations

# Pretvorba podataka u matricu ocjena korisnika
user_movie_ratings = ratings_data.pivot(index='userId', columns='movieId', values='rating').fillna(0)

# Pronalaženje sličnih korisnika za određenog korisnika (primjer: korisnik s ID-om 1)
target_user_id = 1
similar_users = find_similar_users(target_user_id, user_movie_ratings)

# Dobivanje preporuka za ciljnog korisnika
recommendations = get_movie_recommendations(target_user_id, user_movie_ratings, similar_users)
print(f'Recommended movies for user {target_user_id}:\n{recommendations}')

In [24]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report

# Pretpostavljeni DataFrame s oznakama za filmove, uključujući i ID korisnika
user_movie_tags = pd.DataFrame({
    'userId': [1, 1, 2, 2, 3, 3, 4, 4],
    'movieId': [101, 102, 101, 103, 102, 104, 105, 106],
    'tag': ['drama', 'comedy', 'action', 'drama', 'drama', 'romance', 'action', 'drama']
})

# Podijela podataka na trening i test skupove
train_data, test_data = train_test_split(user_movie_tags, test_size=0.2, random_state=42)

# Koristimo TfidfVectorizer za pretvorbu teksta oznaka u značajke
tfidf_vectorizer = TfidfVectorizer(stop_words='english', max_features=5000)
classifier = MultinomialNB()

# Kreiramo cjevovod koji uključuje vektorizator i klasifikator
model = Pipeline([('tfidf', tfidf_vectorizer), ('classifier', classifier)])

# Treniramo model na trenirajućim podacima
model.fit(train_data['tag'], train_data['tag'])

# Predvidi oznake na test skupu
predictions = model.predict(test_data['tag'])

# Evaluiraj performanse modela
accuracy = accuracy_score(test_data['tag'], predictions)
print(f'Accuracy: {accuracy}')

# Ispisi detaljan izvještaj o klasifikaciji
print(classification_report(test_data['tag'], predictions))

# Primjer predviđanja novih oznaka za odabrani korisnik
selected_user_tags = ['thriller', 'romantic', 'action']
predicted_tags = model.predict(selected_user_tags)
print(f'Predicted tags for the selected user: {predicted_tags}')


Accuracy: 0.0
              precision    recall  f1-score   support

      comedy       0.00      0.00      0.00       1.0
       drama       0.00      0.00      0.00       0.0
     romance       0.00      0.00      0.00       1.0

    accuracy                           0.00       2.0
   macro avg       0.00      0.00      0.00       2.0
weighted avg       0.00      0.00      0.00       2.0

Predicted tags for the selected user: ['drama' 'drama' 'action']


c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
c:\Users\DT User3\AppData\Local\Programs\Python\Python310\lib\site-packages\sklearn\metrics\_classification.py:1471: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average